In [16]:
import io
import re
import pandas as pd
from os import walk
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [17]:
#path of invoices
mypath = 'C:/zPython/benefits/allocation'

#variable to append invoice name
list_allocation = []
for (dirpath, dirnames, filenames) in walk(mypath):
    list_allocation.extend(filenames)
    break

list_allocation

['ABPI Cigna & Life Disability March 2020.xlsx',
 'CMSI Cigna & Life Disability March 2020.xlsx',
 'DWTG Cigna & Life Disability March 2020.xlsx',
 'MDnA Cigna & Life Disability March 2020.xlsx',
 'MHIA Cigna & Life Disability March 2020.xlsx',
 'MHPS Cigna & Life Disability March 2020.xlsx',
 'MITAC-A Cigna & Life Disability March 2020.xlsx',
 'MNES Cigna & Life Disability March 2020.xlsx',
 'Mohawk Cigna & Life Disability March 2020.xlsx',
 'MSSA Cigna & Life Disability March 2020.xlsx',
 'PWPS Cigna & Life Disability March 2020.xlsx']

In [18]:
#loop
invoice_allocation = pd.DataFrame()

for x in list_allocation:
    
    #file name
    invoice_dir = x
    #full directory + file name
    invoice_full_dir = mypath + "/" + invoice_dir

    #read excel file information from titles to last row 11:-1 and last column [0,-1]
    df_invoice_alloc = pd.read_excel(invoice_full_dir).iloc[11:-1,[0,-1]]
    #affiliate name
    affiliate = str(invoice_dir[0:4])
    #name columns
    df_invoice_alloc.rename(columns={ df_invoice_alloc.columns[0]: "Cost_Center" }, inplace = True)
    df_invoice_alloc.rename(columns={ df_invoice_alloc.columns[1]: "Amount" }, inplace = True)
    #convert column values to numeric
    df_invoice_alloc["Amount"] = pd.to_numeric(df_invoice_alloc["Amount"])
    #round values
    df_invoice_alloc = df_invoice_alloc.round({"Amount" : 2})
    #add column with percent
    df_invoice_alloc['percent'] = df_invoice_alloc["Amount"]/df_invoice_alloc["Amount"].sum()
    df_invoice_alloc['Affiliate'] = affiliate

    df_invoice_alloc.set_index('Cost_Center', inplace=True)
    #print(df_invoice_alloc)

    newInfo = df_invoice_alloc
    invoice_allocation = invoice_allocation.append(newInfo)

In [19]:
df_CostCenter = pd.read_csv('C:/zPython/benefits/clean_data/LINA_Mapping.csv')
df_affiliate = pd.read_csv('C:/zPython/benefits/clean_data/LINA_affiliate.csv')
df_new = invoice_allocation.join(df_CostCenter.set_index('Cost_Center'), on='Cost_Center')
df_new = df_new.join(df_affiliate.set_index('Affiliate'), on='Affiliate')

indexNames = df_new[ df_new['Allocate'] == 'No' ].index
 
# Delete these row indexes from dataFrame
df_new.drop(indexNames , inplace=True)

In [20]:
export_path = 'C:/zPython/benefits/clean_data/invoices_allocation.csv'
df_new.to_csv(export_path, index=False)